Opinion Mining Project on Amazon Product Review Dataset

Import all the libraries and function from SparkSQL and SparkMLib

In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
#import wget
from pyspark.ml.feature import Bucketizer,RegexTokenizer,StopWordsRemover,CountVectorizer,IDF, NGram, HashingTF, StringIndexer
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression, RandomForestClassificationModel, RandomForestClassifier, GBTClassificationModel, GBTClassifier, NaiveBayes
from pyspark.ml import Pipeline,PipelineModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator


spark = SparkSession.builder.appName(
    "Opinion mining on Amazon Fashion product reviews"
).getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [0]:
#Create UnionAll function to merge dataframs
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)
 

In [0]:
# File location and type for Clothing , Shoes and Jewelry data
file_location = "/FileStore/tables/reviews_Clothing_Shoes_and_Jewelry_5_json.gz"
file_type = "gz"

# The applied options are for CSV files. For other file types, these will be ignored.
clsj = spark.read.json(file_location)

In [0]:
# File location and type for beauty data
file_location = "/FileStore/tables/reviews_Beauty_5_json.gz"
file_type = "gz"

# The applied options are for CSV files. For other file types, these will be ignored.
bt = spark.read.json(file_location)

In [0]:
#Execute union to merge bt and clsj dataframes
df  = unionAll(bt,clsj)

In [0]:
#Drop the columns that are not required part 1
df = df.drop("image")\
    .drop("verified")\
    .drop("vote")
df.columns

Out[49]: ['asin',
 'helpful',
 'overall',
 'reviewText',
 'reviewTime',
 'reviewerID',
 'reviewerName',
 'summary',
 'unixReviewTime']

In [0]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [0]:
# Drop and columns and concatenate summary and reviewtext as Text column
df = df.withColumn("text",concat(F.col("summary"), lit(" "),F.col("reviewText")))\
 .drop("helpful")\
 .drop("reviewerID")\
 .drop("reviewerName")\
 .drop("reviewTime")

In [0]:
#Print the Count of number of instances 
df.count()

Out[53]: 477179

In [0]:
#Describe the Distribution of the ratings column 'overall' 
df.describe("overall").show()

+-------+------------------+
|summary|           overall|
+-------+------------------+
|  count|            477179|
|   mean|4.2223610007984425|
| stddev|1.1306299814841152|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



In [0]:
#Data Cleaning : Finding Null Values in each column
for col in df.columns:
    print(col,":",df[df[col].isNull()].count())
    

asin : 0
overall : 0
reviewText : 0
summary : 0
unixReviewTime : 0
text : 0


In [0]:
#Filter out the  rows with neutral overall ratings
df1 = df.filter("overall !=3")

#Create a split such that from -inf to 4 will be placed in bucket 0 and 4 to inf will be placed in bucket 1
splits = [-float("inf"), 4.0, float("inf")]

#Bucketize data and create labels 0 if overall rating is in (1.0,2.0), #otherwise 1
bucketizer = Bucketizer(splits=splits,\
                        inputCol="overall", outputCol="label")

df2= bucketizer.transform(df1)

df2.groupBy("overall","label").count().show()

+-------+-----+------+
|overall|label| count|
+-------+-----+------+
|    2.0|  0.0| 26919|
|    5.0|  1.0|277771|
|    1.0|  0.0| 21718|
|    4.0|  1.0| 98098|
+-------+-----+------+



In [0]:
#Diplay the dataframe after droping the columns and creating a new target variable/column 'label' based on overall ratings
df2.show()

+----------+-------+--------------------+--------------------+--------------+--------------------+-----+
|      asin|overall|          reviewText|             summary|unixReviewTime|                text|label|
+----------+-------+--------------------+--------------------+--------------+--------------------+-----+
|7806397051|    1.0|Very oily and cre...|Don't waste your ...|    1391040000|Don't waste your ...|  0.0|
|7806397051|    4.0|The texture of th...|       great quality|    1378425600|great quality The...|  1.0|
|7806397051|    2.0|I really can't te...|Do not work on my...|    1386460800|Do not work on my...|  0.0|
|7806397051|    5.0|I was very happy ...|  Very nice palette!|    1365984000|Very nice palette...|  1.0|
|7806397051|    1.0|PLEASE DONT DO IT...|              smh!!!|    1376611200|smh!!! PLEASE DON...|  0.0|
|7806397051|    2.0|Chalky,Not Pigmen...|Chalky, Not Pigme...|    1378252800|Chalky, Not Pigme...|  0.0|
|9759091062|    2.0|Did nothing for m...|no Lightening,

In [0]:
#Sample withought replacement to resolve the imbalance in the target column
fractions = {1.0 : .1, 0.0 : 1.0}
df3 = df2.stat.sampleBy("label", fractions, 36)
df3.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|48637|
|  1.0|37519|
+-----+-----+



In [0]:
#Split data as 80-20% Train and Test dataset
splitSeed = 5043
trainingData, testData = df3.randomSplit([.80, 0.20], splitSeed)

NLP Text Preprocessing, Pipelines and Model Build

In [0]:
#Tokenize the sentence based on the regex pattern 
tokenizer = RegexTokenizer(inputCol="text",outputCol="reviewTokensUf",pattern="\\s+|[,.()\"]")

#Remove Stop Words that do not contribute in any way to our analysis 
stopwords_remover = StopWordsRemover(stopWords=StopWordsRemover.loadDefaultStopWords("english"),inputCol="reviewTokensUf",outputCol="reviewTokens")

#converts word documents to vectors of token counts
cv = CountVectorizer(inputCol="reviewTokens",outputCol="cv",vocabSize=296337)

#IDF model
idf = IDF(inputCol="cv",outputCol="features")

#Logistic Boosted Classifier
lr = LogisticRegression(maxIter=100,regParam=0.02,elasticNetParam=0.3)

#Create a pipeline by combining all the functions we defined above - tokenizer , stopwords_remover, cv, idf, gbtc
steps =  [tokenizer, stopwords_remover, cv, idf, lr]
lr_pipeline = Pipeline(stages=steps)

#fit the training dataset dataset into the pipeline 
model = lr_pipeline.fit(trainingData)

#Obtain the predictions from the model 
predictions = model.transform(testData)


In [0]:
#Call the Binary Classification Evaluator function 
evaluator = BinaryClassificationEvaluator()  
areaUnderROC = evaluator.evaluate(lr_predictions)
print('Test Area Under ROC for Linear Regression model with Count Vectorizer is ' , areaUnderROC)

Test Area Under ROC for Linear Regression model with Count Vectorizer is  0.9398334608052311


Build a Vocabulary of all the relevant tokens and display the co-efficients

In [0]:
#Building the vocabulary to explore the coefficients 
vocabulary = model.stages[2].vocabulary
weights = model.stages[-1].coefficients.toArray()
weights = [float(weight) for weight in weights]

schema = StructType([StructField('word', StringType()),
                     StructField('weight', FloatType())
                     ])
cdf = spark.createDataFrame(zip(vocabulary, weights), schema)

In [0]:
#Shows the top 10 positive sentiment tokens
cdf.orderBy(desc("weight")).show(10)

+-----------+----------+
|       word|    weight|
+-----------+----------+
|      great|  0.500755|
|       love| 0.4556638|
|    perfect|0.30859488|
|       nice|0.28766462|
|      loves|0.28668725|
|comfortable|0.24460426|
|  excellent|0.22886382|
|     great!|0.22694735|
|compliments|0.21397798|
|     highly|0.20946273|
+-----------+----------+
only showing top 10 rows



In [0]:
#Shows the top 10 negative sentiment tokens
cdf.orderBy("weight").show(10)

+--------------+-----------+
|          word|     weight|
+--------------+-----------+
|  disappointed| -0.4081317|
|      returned|-0.31633556|
|         waste|-0.29761943|
| disappointing|-0.26205418|
|          poor|-0.23398674|
|     returning|-0.22233732|
| unfortunately|-0.22159778|
|        return|-0.22129418|
|disappointment|-0.20847303|
|         cheap|-0.19569387|
+--------------+-----------+
only showing top 10 rows



In [0]:
#model evaluation and metrics for Logistic Regression  vanila version
lp = predictions.select("label", "prediction")
counttotal = predictions.count()
correct = lp.filter(F.col("label")== F.col("prediction")).count()
wrong = lp.filter(~(F.col("label") == F.col("prediction"))).count()
ratioWrong = float(wrong) / float(counttotal)
ratioCorrect=correct/counttotal



trueneg =( lp.filter(F.col("label") == 0.0).filter(F.col("label") == F.col("prediction")).count()) /counttotal
truepos = (lp.filter(F.col("label") == 1.0).filter(F.col("label") == F.col("prediction")).count())/counttotal
falseneg = (lp.filter(F.col("label") == 0.0).filter(~(F.col("label") == F.col("prediction"))).count())/counttotal
falsepos = (lp.filter(F.col("label") == 1.0).filter(~(F.col("label") == F.col("prediction"))).count())/counttotal

precision = truepos / (truepos + falsepos)
recall  = truepos / (truepos + falseneg)
#fmeasure= 2  precision  recall / (precision + recall)
accuracy=(truepos + trueneg) / (truepos + trueneg + falsepos + falseneg)

print('counttotal   :', counttotal     )
print('correct      :', correct        )
print('wrong        :', wrong          )
print('ratioWrong   :', ratioWrong     )
print('ratioCorrect :', ratioCorrect   )
print('truen        :', trueneg          )
print('truep        :', truepos          )
print('falsen       :', falseneg         )
print('falsep       :', falsepos         )
print('precision    :', precision      )
print('recall       :', recall         )
print('accuracy     :', accuracy       )

counttotal   : 17244
correct      : 14993
wrong        : 2251
ratioWrong   : 0.1305381581999536
ratioCorrect : 0.8694618418000464
truen        : 0.518905126420784
truep        : 0.35055671537926236
falsen       : 0.050916260728369286
falsep       : 0.07962189747158432
precision    : 0.8149096791588029
recall       : 0.8731763686263181
accuracy     : 0.8694618418000464


In [0]:
# Display the columns of predictions dataframe after transformation
predictions.printSchema()
display(predictions)

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- text: string (nullable = true)
 |-- label: double (nullable = true)
 |-- reviewTokensUf: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- reviewTokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- cv: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



asin overall reviewText summary unixReviewTime text label reviewTokensUf reviewTokens cv features rawPrediction probability prediction 9759091062 1.0 Did nothing for my skin. Used as suggested and no signs of brightness. Wish it worked for me, but it didn't. Nothing 1392681600 Nothing Did nothing for my skin. Used as suggested and no signs of brightness. Wish it worked for me, but it didn't. 0.0 List(nothing, did, nothing, for, my, skin, used, as, suggested, and, no, signs, of, brightness, wish, it, worked, for, me, but, it, didn't) List(nothing, nothing, skin, used, suggested, signs, brightness, wish, worked) Map(vectorType -> sparse, length -> 87097, indices -> List(8, 22, 122, 240, 298, 1695, 3030, 8018), values -> List(1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0)) Map(vectorType -> sparse, length -> 87097, indices -> List(8, 22, 122, 240, 298, 1695, 3030, 8018), values -> List(2.2293398638773105, 2.2142154371841274, 6.22103204836356, 3.565528418941782, 3.7939449552728037, 5.774624103427492, 6.651963748717203, 8.250228360553178)) Map(vectorType -> dense, length -> 2, values -> List(1.390327565625129, -1.390327565625129)) Map(vectorType -> dense, length -> 2, values -> List(0.80064453198834, 0.19935546801166004)) 0.0 B00004TMFE 2.0 I was so very disappointed with my purchase of the Avalon Thickening shampoo and conditioner. I had such high hopes, but in only two shampooings, my hair was noticeably stripped of oils, had that "squeaky clean" feeling you get when using a very harsh product, and it became dry and brittle. It started looking very dull. They use many products that have an oil stripping effect on the hair, like orange oil and lemon oil. You might think because it says, 'oil' that it might add to your hair's shine, but these are ingredients in industrial strength degreasers. This product only damaged my hair. Avalon is only 70% organic and it had a rather unpleasant smell to me. Rather like mosquito repellant or dish soap. Using a harsh product like this is very detrimental for your hair. I know that a lot of people will be drawn to Avalon because of the low price, but if it drastically stripped my extremely oily hair in two shampoos, I can't imagine what it might do to yours. Although my hair is very fine, so perhaps ethnic girls, and women with coarser hair might benefit? I honestly wouldn't recommend it. I'd rather see people get something that will improve their self-esteem immediately, rather than give them new problems to worry about.In desperation I purchased the more expensive Just Naturals system for thinning hair and believe me, it blows Avalon out of the water. In just one shampoo my hair regained it's texture and became so silky it felt like Asian hair. I can't stop touching my hair now and I feel so much more confident. They use incredible ingredients that greatly benefit people with hair loss due to hormonal issues, or who have an increase in DHT. You do pay more than double for that shampoo, but having something that supports your hair, improves the texture, and smells fantastic, was worth it to me. I'm now quite confident in what I'm using and plan to go on using it, regardless of the cost. I'd pay 5x what I'm paying for it now, I love it so much. And their treatment spray is oil based. I use the system twice a day and I'm shocked at how long it is lasting. It has truly been an excellent investment.I would recommend that you shampoo and condition your hair twice a day if you want to see regrowth with any system. This is because one application of shampoo with DHT-fighting ingredients, can only do so much. Your body is constantly generating new hormones and new DHT. By helping to combat it twice a day, you have a better chance of seeing results sooner. This is coming from a 36 year old woman who has been fighting repeated hair loss due to hormonal imbalance from 16 on. Once a day just never cut it for me. Using a leave on treatment like Just Naturals has overnight, can also work wonders. I use it during th

In [0]:
#Model Logistic Regrssion with Bigrams and Hashing TF with IDF vectorization

#Create a pipeline by combining all the functions we defined above - tokenizer , stopwords_remover, cv, idf, gbtc

#Tokenize the sentence based on the regex pattern 
tokenizer = RegexTokenizer(inputCol="text",outputCol="reviewTokensUf",pattern="\\s+|[,.()\"]")


bigram = NGram(inputCol = "reviewTokensUf", outputCol = "bigrams", n = 2)


tfs  = HashingTF(inputCol="bigrams", outputCol="h_features")

#IDF model
idf = IDF(inputCol="h_features",outputCol="features")

lr = LogisticRegression(maxIter=20)

steps =  [tokenizer, stopwords_remover, bigram, tfs, idf, lr]
bigrams_pipeline = Pipeline(stages=steps)

model = bigrams_pipeline.fit(trainingData)
bi_predictions = model.transform(testData)

In [0]:
evaluator = BinaryClassificationEvaluator()  
areaUnderROC = evaluator.evaluate(bi_predictions)
print('Test Area Under ROC for Bigrams linear regression', areaUnderROC)

Test Area Under ROC for Bigrams linear regression 0.9467960427315577


In [0]:
#model evaluation
lp = bi_predictions.select("label", "prediction")
counttotal = bi_predictions.count()
correct = lp.filter(F.col("label")== F.col("prediction")).count()
wrong = lp.filter(~(F.col("label") == F.col("prediction"))).count()
ratioWrong = float(wrong) / float(counttotal)
ratioCorrect=correct/counttotal



trueneg =( lp.filter(F.col("label") == 0.0).filter(F.col("label") == F.col("prediction")).count()) /counttotal
truepos = (lp.filter(F.col("label") == 1.0).filter(F.col("label") == F.col("prediction")).count())/counttotal
falseneg = (lp.filter(F.col("label") == 0.0).filter(~(F.col("label") == F.col("prediction"))).count())/counttotal
falsepos = (lp.filter(F.col("label") == 1.0).filter(~(F.col("label") == F.col("prediction"))).count())/counttotal

precision = truepos / (truepos + falsepos)
recall  = truepos / (truepos + falseneg)
#fmeasure= 2  precision  recall / (precision + recall)
accuracy=(truepos + trueneg) / (truepos + trueneg + falsepos + falseneg)

print('counttotal   :', counttotal     )
print('correct      :', correct        )
print('wrong        :', wrong          )
print('ratioWrong   :', ratioWrong     )
print('ratioCorrect :', ratioCorrect   )
print('truen        :', trueneg          )
print('truep        :', truepos          )
print('falsen       :', falseneg         )
print('falsep       :', falsepos         )
print('precision    :', precision      )
print('recall       :', recall         )
#print('fmeasure     :', fmeasure       )
print('accuracy     :', accuracy       )

counttotal   : 17244
correct      : 15314
wrong        : 1930
ratioWrong   : 0.11192298770586871
ratioCorrect : 0.8880770122941313
truen        : 0.5028995592669914
truep        : 0.38517745302713985
falsen       : 0.06692182788216192
falsep       : 0.045001159823706796
precision    : 0.8953895928821785
recall       : 0.8519753719856337
accuracy     : 0.8880770122941313


In [0]:
# BiGRams created after the transformation
bi_predictions.select(F.col("bigrams")).show(5)

+--------------------+
|             bigrams|
+--------------------+
|[nothing did, did...|
|[industrial stren...|
|[dries out, out m...|
|[not impressed, i...|
|[love this, this ...|
+--------------------+
only showing top 5 rows



d
RandomForest Implementation

In [0]:
#RandomForest Implementation 
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

#Remove Stop Words that do not contribute in any way to our analysis 
stopwords_remover_rf = \
StopWordsRemover(stopWords=StopWordsRemover.loadDefaultStopWords("english"),inputCol="words",outputCol="filtered")

# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)


label_stringIdx = StringIndexer(inputCol = "label", outputCol = "new_label")

rf = RandomForestClassifier(numTrees=3, maxDepth=2, labelCol="new_label", seed=42, \
                            leafCol="leafId")

rf = RandomForestClassifier(labelCol="new_label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

steps =  [regexTokenizer, stopwords_remover_rf, countVectors, label_stringIdx, rf]
randomforests_pipeline = Pipeline(stages=steps)

odel = randomforests_pipeline.fit(trainingData)
rf_predictions = model.transform(testData)

In [0]:
rf_predictions.printSchema()
rf_predictions.show(5)

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- text: string (nullable = true)
 |-- label: double (nullable = true)
 |-- reviewTokensUf: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- reviewTokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bigrams: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- h_features: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [0]:
evaluator = BinaryClassificationEvaluator()  
areaUnderROC = evaluator.evaluate(rf_predictions)
print('Test Area Under ROC for Random Forests Classification is ', areaUnderROC)

Test Area Under ROC for Random Forests Classification is  0.8941074905567711


In [0]:
y_pred=rf_predictions.select("prediction").collect()
y_orig=rf_predictions.select("label").collect()

cf = classification_report(y_pred, y_orig)
print(cf)


              precision    recall  f1-score   support

         0.0       0.88      0.92      0.90      9448
         1.0       0.90      0.85      0.87      7796

    accuracy                           0.89     17244
   macro avg       0.89      0.88      0.89     17244
weighted avg       0.89      0.89      0.89     17244



In [0]:
#model evaluation
lp = rf_predictions.select("label", "prediction")
counttotal = rf_predictions.count()
correct = lp.filter(F.col("label")== F.col("prediction")).count()
wrong = lp.filter(~(F.col("label") == F.col("prediction"))).count()
ratioWrong = float(wrong) / float(counttotal)
ratioCorrect=correct/counttotal



trueneg =( lp.filter(F.col("label") == 0.0).filter(F.col("label") == F.col("prediction")).count()) /counttotal
truepos = (lp.filter(F.col("label") == 1.0).filter(F.col("label") == F.col("prediction")).count())/counttotal
falseneg = (lp.filter(F.col("label") == 0.0).filter(~(F.col("label") == F.col("prediction"))).count())/counttotal
falsepos = (lp.filter(F.col("label") == 1.0).filter(~(F.col("label") == F.col("prediction"))).count())/counttotal

precision = truepos / (truepos + falsepos)
recall  = truepos / (truepos + falseneg)
#fmeasure= 2  precision  recall / (precision + recall)
accuracy=(truepos + trueneg) / (truepos + trueneg + falsepos + falseneg)

print('counttotal   :', counttotal     )
print('correct      :', correct        )
print('wrong        :', wrong          )
print('ratioWrong   :', ratioWrong     )
print('ratioCorrect :', ratioCorrect   )
print('truen        :', trueneg          )
print('truep        :', truepos          )
print('falsen       :', falseneg         )
print('falsep       :', falsepos         )
print('precision    :', precision      )
print('recall       :', recall         )
#print('fmeasure     :', fmeasure       )
print('accuracy     :', accuracy       )

counttotal   : 17244
correct      : 10028
wrong        : 7216
ratioWrong   : 0.4184643934122013
ratioCorrect : 0.5815356065877987
truen        : 0.5695314312224542
truep        : 0.012004175365344467
falsen       : 0.00028995592669914174
falsep       : 0.4181744374855022
precision    : 0.027905095713130223
recall       : 0.9764150943396226
accuracy     : 0.5815356065877987


Naive Bayes

In [0]:
nb = NaiveBayes(smoothing=1)
steps =  [regexTokenizer, stopwords_remover_rf, countVectors, label_stringIdx, nb]

nb_pipeline = Pipeline(stages=steps)
model = nb_pipeline.fit(trainingData)
nb_predictions = model.transform(testData)

In [0]:
evaluator = BinaryClassificationEvaluator()  
areaUnderROC = evaluator.evaluate(nb_predictions)
print('Test Area Under ROC for Naive Bayes Classification is ', areaUnderROC)

Test Area Under ROC for Naive Beyes Classification is  0.5399622753791408


In [0]:
#model evaluation
lp = nb_predictions.select("label", "prediction")
counttotal = nb_predictions.count()
correct = lp.filter(F.col("label")== F.col("prediction")).count()
wrong = lp.filter(~(F.col("label") == F.col("prediction"))).count()
ratioWrong = float(wrong) / float(counttotal)
ratioCorrect=correct/counttotal



trueneg =( lp.filter(F.col("label") == 0.0).filter(F.col("label") == F.col("prediction")).count()) /counttotal
truepos = (lp.filter(F.col("label") == 1.0).filter(F.col("label") == F.col("prediction")).count())/counttotal
falseneg = (lp.filter(F.col("label") == 0.0).filter(~(F.col("label") == F.col("prediction"))).count())/counttotal
falsepos = (lp.filter(F.col("label") == 1.0).filter(~(F.col("label") == F.col("prediction"))).count())/counttotal

precision = truepos / (truepos + falsepos)
recall  = truepos / (truepos + falseneg)
#fmeasure= 2  precision  recall / (precision + recall)
accuracy=(truepos + trueneg) / (truepos + trueneg + falsepos + falseneg)

print('counttotal   :', counttotal     )
print('correct      :', correct        )
print('wrong        :', wrong          )
print('ratioWrong   :', ratioWrong     )
print('ratioCorrect :', ratioCorrect   )
print('truen        :', trueneg          )
print('truep        :', truepos          )
print('falsen       :', falseneg         )
print('falsep       :', falsepos         )
print('precision    :', precision      )
print('recall       :', recall         )
#print('fmeasure     :', fmeasure       )
print('accuracy     :', accuracy       )

counttotal   : 17244
correct      : 15240
wrong        : 2004
ratioWrong   : 0.116214335421016
ratioCorrect : 0.883785664578984
truen        : 0.5045813036418464
truep        : 0.37920436093713755
falsen       : 0.0652400835073069
falsep       : 0.050974251913709114
precision    : 0.881504448638447
recall       : 0.8532098121085595
accuracy     : 0.883785664578984


In [0]:
#GBTClassifier : Gradiennt Boosted Trees

#Tokenize the sentence based on the regex pattern 
tokenizer = RegexTokenizer(inputCol="text",outputCol="reviewTokensUf",pattern="\\s+|[,.()\"]")


#Remove Stop Words that do not contribute in any way to our analysis 
stopwords_remover = StopWordsRemover(stopWords=StopWordsRemover.loadDefaultStopWords("english"),inputCol="reviewTokensUf",outputCol="reviewTokens")

#converts word documents to vectors of token counts
cv = CountVectorizer(inputCol="reviewTokens",outputCol="cv",vocabSize=296337)



steps =  [tokenizer, stopwords_remover, cv]
Pipeline_mo = Pipeline(stages=steps)
transformed_model = Pipeline_mo.fit(trainingData)
train_tr = transformed_model.transform(trainingData)
test_tr = transformed_model.transform(testData)


In [0]:
train_tr.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- text: string (nullable = true)
 |-- label: double (nullable = true)
 |-- reviewTokensUf: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- reviewTokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- cv: vector (nullable = true)



In [0]:
train_tr.show(5)

+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+
|      asin|overall|          reviewText|             summary|unixReviewTime|                text|label|      reviewTokensUf|        reviewTokens|                  cv|
+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+
|7806397051|    1.0|PLEASE DONT DO IT...|              smh!!!|    1376611200|smh!!! PLEASE DON...|  0.0|[smh!!!, please, ...|[smh!!!, please, ...|(87097,[0,4,5,9,4...|
|7806397051|    1.0|Very oily and cre...|Don't waste your ...|    1391040000|Don't waste your ...|  0.0|[don't, waste, yo...|[waste, money, oi...|(87097,[43,55,67,...|
|7806397051|    2.0|Chalky,Not Pigmen...|Chalky, Not Pigme...|    1378252800|Chalky, Not Pigme...|  0.0|[chalky, not, pig...|[chalky, pigmente...|(87097,[46,103

In [0]:
# training  
gbtc = GBTClassifier(featuresCol="cv", labelCol="label", maxIter=20)
gbtc = gbtc.fit(train_tr)

# prediction
pred = gbtc.transform(test_tr)
pred.show(3) 

+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      asin|overall|          reviewText|             summary|unixReviewTime|                text|label|      reviewTokensUf|        reviewTokens|                  cv|       rawPrediction|         probability|prediction|
+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|9759091062|    1.0|Did nothing for m...|             Nothing|    1392681600|Nothing Did nothi...|  0.0|[nothing, did, no...|[nothing, nothing...|(87097,[8,22,122,...|[0.59522853597419...|[0.76682280121619...|       0.0|
|B00004TMFE|    2.0|I was so very dis...|Industrial streng...|    1401840000|Industrial streng...|  0.0|[industrial,

In [0]:
print(gbtc)

GBTClassificationModel: uid = GBTClassifier_b7bb4030176d, numTrees=20, numClasses=2, numFeatures=87097


In [0]:
evaluator = BinaryClassificationEvaluator()  
areaUnderROC = evaluator.evaluate(pred)
print('Test Area Under ROC for Gradient Boost Classification is ', areaUnderROC)

Test Area Under ROC for Naive Bayes Classification is  0.8692959846983227


In [0]:
pred.show(5)

+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      asin|overall|          reviewText|             summary|unixReviewTime|                text|label|      reviewTokensUf|        reviewTokens|                  cv|       rawPrediction|         probability|prediction|
+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|9759091062|    1.0|Did nothing for m...|             Nothing|    1392681600|Nothing Did nothi...|  0.0|[nothing, did, no...|[nothing, nothing...|(87097,[8,22,122,...|[0.59522853597419...|[0.76682280121619...|       0.0|
|B00004TMFE|    2.0|I was so very dis...|Industrial streng...|    1401840000|Industrial streng...|  0.0|[industrial,

In [0]:
from sklearn.metrics import confusion_matrix, classification_report

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator=MulticlassClassificationEvaluator(metricName="accuracy")
#e
acc = evaluator.evaluate(pred)
 
print("Prediction Accuracy: ", acc)

y_pred=pred.select("prediction").collect()
y_orig=pred.select("label").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm) 



Prediction Accuracy:  0.7811992577128276
Confusion Matrix:
[[8332 1494]
 [2279 5139]]


In [0]:
from sklearn.metrics import confusion_matrix, classification_report

cf = classification_report(y_orig, y_pred)
print(cf)

              precision    recall  f1-score   support

         0.0       0.79      0.85      0.82      9826
         1.0       0.77      0.69      0.73      7418

    accuracy                           0.78     17244
   macro avg       0.78      0.77      0.77     17244
weighted avg       0.78      0.78      0.78     17244

